In [1]:
!pwd
%cd /home/thucth/Biometrics/insightface/recognition/arcface_torch

import os
#choose CUDA devices
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "2"


/home/thucth/Biometrics/insightface/recognition/arcface_torch
/home/thucth/Biometrics/insightface/recognition/arcface_torch


In [10]:
import argparse
import cv2
import numpy as np
import torch

from backbones import get_model

@torch.no_grad()
def inference(model, img_path):
    if img_path is None:
        img = np.random.randint(0, 255, size=(112, 112, 3), dtype=np.uint8)
    else:
        img = cv2.imread(img_path)
        assert img.shape[1] == 112

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.transpose(img, (2, 0, 1))
    img = torch.from_numpy(img).unsqueeze(0).float()
    img.div_(255).sub_(0.5).div_(0.5)
    
    emb, norm = model(img)
    return emb

@torch.no_grad()
def inference_batch(model, img_paths=None):
    imgs = []
    if img_paths is None:
        img = np.random.randint(0, 255, size=(3, 112, 112), dtype=np.uint8)
    else:
        for img_path in img_paths:
            img = cv2.imread(img_path)
            # img = img[:,:,::-1]#convert BGR to RGB
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = np.transpose(img, (2, 0, 1))
            img = torch.from_numpy(img).float()
            img.div_(255).sub_(0.5).div_(0.5)
            
            assert img.shape[0] == 3
            assert img.shape[1] == 112
            imgs.append(img)

    device = next(model.parameters()).device
    imgs = torch.stack(imgs).to(device).float()
    embs, norms = model(imgs)

    return embs



In [151]:
[f"cuda:{i}" for i in [0,1,2,3,4,5]]

['cuda:0', 'cuda:1', 'cuda:2', 'cuda:3', 'cuda:4', 'cuda:5']

In [3]:
print(devices)


[device(type='cuda', index=0), device(type='cuda', index=1), device(type='cuda', index=2), device(type='cuda', index=3), device(type='cuda', index=4), device(type='cuda', index=5)]


In [18]:
import torch.nn as nn 


net = get_model("ir_101", fp16=False)
#1.422179
# net.load_state_dict(torch.load("/home/thucth/Biometrics/insightface/recognition/arcface_torch/work_dirs/model_3.pt", map_location=torch.device('cuda')))
net.load_state_dict(torch.load("/home/thucth/Biometrics/insightface/recognition/arcface_torch/work_dirs/finetune_363kIden_1306kImage/model_10.pt", map_location=torch.device('cuda')))
net.eval()
net.cuda()
# devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]

net = nn.DataParallel(net)

In [ ]:
net1 = get_model("ir_101", fp16=False)
net1.load_state_dict(torch.load("/home/thucth/Biometrics/insightface/recognition/arcface_torch/work_dirs/finetune_363kIden_1306kImage/model_10.pt", map_location=torch.device('cuda')))
net1.eval()
net1.to("cuda:2")

In [12]:
net3 = get_model("ir_101", fp16=False)
net3.load_state_dict(torch.load("/home/thucth/Biometrics/insightface/recognition/arcface_torch/work_dirs/finetune_363kIden_1306kImage/model_10.pt", map_location=torch.device('cuda')))
net3.eval()
net3.to("cuda:3")

Backbone(
  (input_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (output_layer): Sequential(
    (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.4, inplace=False)
    (2): Flatten()
    (3): Linear(in_features=25088, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  )
  (body): Sequential(
    (0): BasicBlockIR(
      (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
      (res_layer): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1

In [ ]:
output_single = inference(net, img_path="/home/thucth/Biometrics/Internal/InternalAligned/0/210419000241095_1_265407.jpg")
output_single[0,:10]

In [14]:
from glob import glob 

def find_outlier(net, cluster_path, threshold = 1.422179/2):
    img_paths = glob(os.path.join(cluster_path,"*"))
    embs = inference_batch(net, img_paths=img_paths)
    embs = embs.cpu()
    centroid = embs.mean(dim=0, keepdim=True)
    distances = np.linalg.norm(embs - centroid, axis=1)

    outliers = np.where(distances > threshold)[0]
    return cluster_path, [(img_paths[i],distances[i]) for i in outliers.tolist()]

In [ ]:
import matplotlib.pyplot as plt 
from tqdm import tqdm 

for cluster_path in tqdm(glob(os.path.join("/home/thucth/Biometrics/Internal/InternalAligned/*"))):
        
    # result_outliers = find_outlier(cluster_path, threshold = 1.612923/2)
    result_outliers = find_outlier(net, cluster_path, threshold = 1.2)

    if len(result_outliers[1]) == 0:
        continue
    
    print("Outliers")
    outliers = {outlier:1 for outlier in result_outliers[1]}
    for idx, outlier in enumerate(outliers):
        plt.subplot(1,len(outliers),idx+1)
        plt.imshow(cv2.imread(outlier[0])[:,:,::-1])
        plt.title(outlier[1])
    plt.show()

    print("Normals")
    normals = []
    for path in glob(result_outliers[0]+"/*"):
        if path not in [k[0] for k in outliers.keys()]:
            normals.append(path)
    for idx, img_path in enumerate(normals):
        plt.subplot(2,len(normals),idx+1)
        plt.imshow(cv2.imread(img_path)[:,:,::-1])
    plt.show()        

In [ ]:
def filter_noise(model, cluster_paths, output = "./outliers.csv", threshold = 1.4):
    with open(output, "w") as f:
        csv_writer = csv.writer(f)
        csv_writer.write(["path","dis_to_centroid"])
        for cluster_path in cluster_paths:
            result_outliers = find_outlier(model, cluster_path, threshold = threshold)

            if len(result_outliers[1]) == 0:
                continue
            
            outliers = {outlier:1 for outlier in result_outliers[1]}
            for idx, outlier in enumerate(outliers):
                csv_writer.writerow([outlier[0],outlier[1]])